In [ ]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from shutil import copyfile
import numpy as np
import scipy.io
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import glob
import imageio
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import datetime
import random
from google.colab import files

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# copyfile('gdrive/My Drive/dl-prog-pro/immagini2000.npy', 'immagini2000.npy')
# copyfile('gdrive/My Drive/dl-prog-pro/maschere2000.npy', 'maschere2000.npy')

In [ ]:
X_train = np.load('immagini2000.npy')
y_train = np.load('maschere2000.npy')

In [ ]:
num = 1600

In [ ]:
X_train = X_train[0:num,:,:,:]
y_train = y_train[0:num,:,:]

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
OUTPUT_CHANNELS = 6

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256, 256, 1])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
    downsample(128, 4),  # (batch_size, 64, 64, 128)
    downsample(256, 4),  # (batch_size, 32, 32, 256)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
    downsample(512, 4),  # (batch_size, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(256, 4),  # (batch_size, 32, 32, 512)
    upsample(128, 4),  # (batch_size, 64, 64, 256)
    upsample(64, 4),  # (batch_size, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='sigmoid')  # (batch_size, 256, 256, 6)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

    
  skips = reversed(skips[:-1])
  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
num_prova = 1

In [ ]:
generator = Generator()
gen_output = generator(tf.expand_dims(tf.expand_dims(y_train[num_prova,:,:],2),0), training=False)
plt.imshow(gen_output[0,:,:,0:3])

In [ ]:
# tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
LAMBDA = 100

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 6], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
  down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
  down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
# tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
discriminator = Discriminator()
disc_out = discriminator([tf.expand_dims(tf.expand_dims(y_train[num_prova,:,:],2),0), gen_output], training=False)
plt.imshow(disc_out[0,:,:,0])

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
  total_disc_loss = real_loss + generated_loss
  return total_disc_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0,:,:,0], tar[0,:,:,0:3], prediction[0,:,:,0:3]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    if i==1 or i==2:
      plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    else:
      plt.imshow(display_list[i])
    plt.axis('off')
  plt.show()

In [ ]:
example_input = tf.expand_dims(tf.expand_dims(y_train[num_prova,:,:],0),3)
example_target = tf.expand_dims(X_train[num_prova,:,:,:],0)
print(example_input.shape, example_target.shape)
generate_images(generator, example_input, example_target)

In [ ]:
example_input.shape

In [ ]:
@tf.function
def train_step(input_image, target, step):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  # with summary_writer.as_default():
  #   tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//10)
  #   tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//10)
  #   tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//10)
  #   tf.summary.scalar('disc_loss', disc_loss, step=step//10)

In [ ]:
@tf.function
def train_step2(input_image, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [ ]:
def train(X_train, y_train, steps):
  example_input = y_train[:,num_prova,:,:]
  example_target = X_train[:,num_prova,:,:,:]
  start = time.time()
  for step in range(steps):
    i = np.random.choice(np.arange(len(y_train)))
    input_image = y_train[:,i,:,:] 
    target = X_train[:,i,:,:,:]
    if (step) % 100 == 0:
      display.clear_output(wait=True)

      if step != 0:
        print(f'Time taken for {step} steps: {time.time()-start:.2f} sec\n')

      start = time.time()

      generate_images(generator, example_input, example_target)
      print(f"Step: {step}")

    train_step(input_image, target, step)

    # Training step
    if (step+1) % 10 == 0:
      print('.', end='', flush=True)


In [ ]:
def train2(X_train_ds, y_train_ds, epochs=20):
  example_input = y_train_ds[num_prova,:,:,:,:]
  example_target = X_train_ds[num_prova,:,:,:,:]
  start = time.time()
  for ep in range(epochs):
    indexes = np.random.permutation(np.arange(num))

    for index in indexes:
      input_image = y_train_ds[index,:,:,:,:]
      target = X_train_ds[index,:,:,:,:]
      train_step2(input_image, target)

    display.clear_output(wait=True)
    generate_images(generator, example_input, example_target)
    print(f'Time taken for {ep+1} epochs: {time.time()-start:.2f} sec\n')

In [ ]:
'''
def train3(X_train, y_train, epochs=20, batch_size=1):
  length = len(y_train[:,0,0])
  subgroups = length//batch_size
  rest = length%batch_size
  example_input = y_train[num_prova,:,:,:]
  example_target = X_train[num_prova,:,:,:]
  start = time.time()

  for ep in range(epochs+1):
    indexes = np.random.permutation(np.arange(len(y_train[:,0,0])))

    for group in range(subgroups):
      index = indexes[group*batch_size:(group+1)*batch_size]
      input_image = y_train[:,index,:,:]
      target = X_train[:,index,:,:,:]
      train_step2(input_image, target)
    if rest !=0:
      index = indexes[length-rest:]
      input_image = X_train[:,index,:,:,:]
      target = y_train[:,index,:,:]
      train_step2(input_image, target)

    if ep%100 == 0:
      display.clear_output(wait=True)
      generate_images(generator, example_input, example_target)
      print(f'Time taken for {ep} epochs: {time.time()-start:.2f} sec\n')
'''

In [ ]:
X_train1 = tf.expand_dims(tf.cast(X_train,'float32'),1)
y_train1 = tf.expand_dims(tf.cast(tf.expand_dims(y_train,3),'float32'),1)

In [ ]:
print(X_train1.shape,y_train1.shape)

In [ ]:
epochs = 20
batch_size = 16
steps=40000

In [ ]:
train2(X_train1, y_train1, epochs = epochs)

In [ ]:
json_model = generator.to_json()

In [ ]:
with open('generator_finale.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
generator.save_weights('generator_finale_weights.h5')

In [ ]:
!cp generator_finale.json "gdrive/My Drive/dl-prog-pro/"
!cp generator_finale_weights.h5 "gdrive/My Drive/dl-prog-pro/"

In [ ]:
with open('generator_finale.json', 'r') as json_file:
    json_savedModel = json_file.read()
net = tf.keras.models.model_from_json(json_savedModel)
net.load_weights('generator_finale_weights.h5')

In [ ]:
num_prova = 2
example_input = tf.expand_dims(tf.expand_dims(y_train[num_prova,:,:],0),3)
example_target = tf.expand_dims(X_train[num_prova,:,:,:],0)
print(example_input.shape, example_target.shape)
generate_images(net, example_input, example_target)